<a href="https://colab.research.google.com/github/itzahs/GeoPythonWorkshop2023_SSL/blob/main/2_GetRepository_SSL_for_aerial_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GeoPython 2023 Workshop

## Section 1: Download the UCM dataset & understand augmentations
## Section 2: Implementation of the FixMatch algorithm using PyTorch 

### Import required packages and libraries 


In [ ]:
# Import 
import os
from google.colab import drive

### Monitor GPU usage

In [ ]:
#  Monitor the GPU usage 
!nvidia-smi

Sun Mar  5 20:13:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    23W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Set up working folder

In [ ]:
# Mount google drive
drive.mount('/content/drive')

# Create working folder 
folder_name = "GeoPythonWorkshop_SSL"
path_to_folder = "/content/drive/MyDrive/" + folder_name

if not os.path.exists(path_to_folder):
    os.makedirs(path_to_folder)
    print(F"Folder '{folder_name}' has been created successfully.")
else:
    print(F"Folder '{folder_name}' already exists.")

# Set working folder as default
%cd "/content/drive/MyDrive/GeoPythonWorkshop_SSL"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder 'GeoPythonWorkshop_SSL' already exists.
/content/drive/MyDrive/GeoPythonWorkshop_SSL


### Clone repository

In [ ]:
!git clone https://github.com/TencentYoutuResearch/Classification-SemiCLS.git

fatal: destination path 'Classification-SemiCLS' already exists and is not an empty directory.


In [ ]:
# Set cloned repository as default
%cd "/content/drive/MyDrive/GeoPythonWorkshop_SSL/Classification-SemiCLS"

# Checking that repository was correctly cloned
!ls

/content/drive/MyDrive/GeoPythonWorkshop_SSL/Classification-SemiCLS
configs  loss	 optimizer	    requirements.txt  tools	     utils
dataset  mmcv	 pretrained_models  results	      trainer
LICENSE  models  README.md	    scheduler	      train_semi.py


### Install all the requirements

In [ ]:
!git clone https://github.com/open-mmlab/mmcv.git
%cd mmcv/
!pip install -e .

fatal: destination path 'mmcv' already exists and is not an empty directory.
/content/drive/MyDrive/GeoPythonWorkshop_SSL/Classification-SemiCLS/mmcv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/GeoPythonWorkshop_SSL/Classification-SemiCLS/mmcv
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmcv
    Found existing installation: mmcv 1.7.1
    Can't uninstall 'mmcv'. No files were found to uninstall.
  Running setup.py develop for mmcv


In [ ]:
# Install other libraries
!pip install torch torchvision torchaudio
!pip install apex
!pip install tensorboardX
!pip install tensorboard
!pip install tensorrt
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Code modifications





#### Config 
The config code defines a configuration for training a deep learning model using the FixMatch algorithm on the UCM dataset. The model architecture used is a wideresnet28x2. The training will be performed on a single GPU with a batch size of 8. Only 4 labeled samples per class will be used for training.
The training process is organized into three main components:  

*   Train: Algorithm, number of steps and loss function.
*   Model: Architecture of the model to be trained. 
*   Data:  Loading and preprocessing the UCM dataset, number of labeled samples, batch size and data augmentation pipeline. 

Other options such as the learning rate scheduler, exponential moving average (EMA) of model parameters, automatic mixed precision (AMP) optimization, and logging and checkpointing options are also defined.

Finally, an optimizer of type SGD is defined with a learning rate of 0.03, momentum of 0.9, weight decay of 0.001, and Nesterov momentum enabled.

#### Dataset builder and training syntax 

For the code to work in a Jupyter notebook environment we need to make two modifications. First, Jupyter notebook creates .ipynb_checkpoints and we need to ignore them and then, make some changes in the code syntax.
1. In dataset/builder.py (lines 15): Add import os
2. In dataset/builder.py (lines 82-92): add check to ignore .ipynb_checkpoints.
3. In train_semi.py (lines 444 & 450): Modify labeled.next() for next(labeled).
4. In train_semi.py (lines 453 & 459): Modify unlabeled.next for next(unlabeled).


### Write automatically the changes in:
1. ./Classification-SemiCLS/configs/fm_ucm.py
2. ./Classification-SemiCLS/dataset/builder.py
3. ./Classification-SemiCLS/train_semi.py. 


In [ ]:
# Set working folder as default
%cd "/content/drive/MyDrive/GeoPythonWorkshop_SSL"

/content/drive/MyDrive/GeoPythonWorkshop_SSL


In [ ]:
%%writefile ./Classification-SemiCLS/configs/fm_ucm.py

""" The Code is under Tencent Youtu Public Rule"""

train = dict(eval_step=1024,
             total_steps=1024*20,
             trainer=dict(type="FixMatch",
                          threshold=0.95,
                          T=1.,
                          lambda_u=1.,
                          loss_x=dict(
                              type="cross_entropy",
                              reduction="mean"),
                          loss_u=dict(
                              type="cross_entropy",
                              reduction="none"),
                          ))
num_classes = 21

model = dict(
     type="wideresnet",
     depth=28,
     widen_factor=2,
     dropout=0,
     num_classes=num_classes,
)

ucm_mean = (0.485, 0.456, 0.406) 
ucm_std = (0.229, 0.224, 0.225)

data = dict(
    type="MyDataset",
    num_workers=0,
    num_labeled=84,
    num_classes=num_classes,
    batch_size=8,
    expand_labels=False,
    mu=7,

    root="./UCMerced_LandUse/Images",
    labeled_names_file="./UCMerced_LandUse/Images/UCM_train.txt",
    test_names_file="./UCMerced_LandUse/Images/UCM_test.txt",
    lpipelines=[[
        dict(type="RandomHorizontalFlip"),
        dict(type="RandomResizedCrop", size=224, scale=(0.2, 1.0)),
        dict(type="ToTensor"),
        dict(type="Normalize", mean=ucm_mean, std=ucm_std)
    ]],
    upipelinse=[[
        dict(type="RandomHorizontalFlip"),
        dict(type="Resize", size=256),
        dict(type="CenterCrop", size=224),
        dict(type="ToTensor"),
        dict(type="Normalize", mean=ucm_mean, std=ucm_std)
        ],
        [
        dict(type="RandomHorizontalFlip"),
        dict(type="RandomResizedCrop", size=224, scale=(0.2, 1.0)),
        dict(type="RandAugmentMC", n=2, m=10),
        dict(type="ToTensor"),
        dict(type="Normalize", mean=ucm_mean, std=ucm_std)
    ]],
    vpipeline=[
        dict(type="Resize", size=256),
        dict(type="ToTensor"),
        dict(type="Normalize", mean=ucm_mean, std=ucm_std)
    ])

scheduler = dict(
    type='cosine_schedule_with_warmup',
    num_warmup_steps=0,
    num_training_steps=train['total_steps']
)

ema = dict(use=True, pseudo_with_ema=False, decay=0.999)
#apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
#"See details at https://nvidia.github.io/apex/amp.html
amp = dict(use=False, opt_level="O1")

log = dict(interval=1)
ckpt = dict(interval=1000)

# optimizer
optimizer = dict(type='SGD', lr=0.03, momentum=0.9, weight_decay=0.001, nesterov=True)

Overwriting ./Classification-SemiCLS/configs/fm_ucm.py


In [ ]:
%%writefile ./Classification-SemiCLS/dataset/builder.py

""" The Code is under Tencent Youtu Public Rule
This file build a dataset for semi supervised learning
dataset_cfg:
    num_labeled:
    num_classes:
    batch_size:
    eval_step:

    root:
    type: cifar10
    lpipelines:
    upipelinse:
    vpipeline:
"""
import os
from torchvision import datasets

from dataset.cifar import CIFAR10SSL, CIFAR100SSL, x_u_split
from dataset.imagenet import get_imagenet_ssl_dataset
from dataset.MyDataset import MyDataset
from dataset.stl10 import get_stl10
from dataset.transforms.builder import BaseTransform, ListTransform
from dataset.txt_dataset import get_txt_ssl_dataset

dataset_dict = {"CIFAR10SSL": CIFAR10SSL, "CIFAR100SSL": CIFAR100SSL}

base_dict = {
    "CIFAR10": datasets.CIFAR10,
    "CIFAR10SSL": datasets.CIFAR10,
    "CIFAR100": datasets.CIFAR100,
    "CIFAR100SSL": datasets.CIFAR100,
    "MyDataset": MyDataset,
}

def build(cfg):

    if cfg.type == "MyDataset":
        transform_ulabeled = ListTransform(cfg.upipelinse)
        train_unlabeled_dataset = datasets.ImageFolder(
            root=cfg.root,
            transform=transform_ulabeled)
        transform_labeled = ListTransform(cfg.lpipelines)
        train_labeled_dataset = MyDataset(names_file=cfg.labeled_names_file, transform=transform_labeled)
        transform_val = BaseTransform(cfg.vpipeline)
        test_dataset = MyDataset(names_file=cfg.test_names_file, transform=transform_val)

    elif cfg.type == "ImagenetSSL":
        transform_labeled = ListTransform(cfg.lpipelines)
        transform_ulabeled = ListTransform(cfg.upipelinse)
        transform_val = BaseTransform(cfg.vpipeline)
        return get_imagenet_ssl_dataset(root=cfg.root,
                                        percent=cfg.percent,
                                        anno_file=cfg.anno_file,
                                        transform_labeled=transform_labeled,
                                        transform_ulabeled=transform_ulabeled,
                                        transform_val=transform_val)

    elif cfg.type == "TxtDatasetSSL":
        transform_labeled = ListTransform(cfg.lpipelines)
        transform_ulabeled = ListTransform(cfg.upipelinse)
        transform_val = BaseTransform(cfg.vpipeline)
        return get_txt_ssl_dataset(
            l_anno_file=cfg.l_anno_file,
            u_anno_file=cfg.u_anno_file,
            v_anno_file=cfg.v_anno_file,
            transform_labeled=transform_labeled,
            transform_ulabeled=transform_ulabeled,
            transform_val=transform_val)

    elif cfg.type == "STL10SSL":
        transform_labeled = ListTransform(cfg.lpipelines)
        transform_ulabeled = ListTransform(cfg.upipelinse)
        transform_val = BaseTransform(cfg.vpipeline)
        return get_stl10(
            root=cfg.root, folds=cfg.folds,
            transform_labeled=transform_labeled,
            transform_ulabeled=transform_ulabeled,
            transform_val=transform_val)

    else:

        # check if .ipynb_checkpoints is in root, and exclude it
        root = os.path.join(cfg.root, '')
        if os.path.isdir(os.path.join(root, ".ipynb_checkpoints")):
            exclude_dir = os.path.join(root, ".ipynb_checkpoints")
        else:
            exclude_dir = None

        base_dataset = base_dict[cfg.type](cfg.root, train=True, download=True)

        train_labeled_idxs, train_unlabeled_idxs = x_u_split(
            cfg, base_dataset.targets)

        # init labeled datasets
        transform_labeled = ListTransform(cfg.lpipelines)
        train_labeled_dataset = dataset_dict[cfg.type](
            cfg.root,
            train_labeled_idxs,
            train=True,
            transform=transform_labeled,
            anno_file=cfg.lanno_file if cfg.get("lanno_file", False) else None)

        transform_ulabeled = ListTransform(cfg.upipelinse)
        train_unlabeled_dataset = dataset_dict[cfg.type](
            cfg.root,
            train_unlabeled_idxs,
            train=True,
            transform=transform_ulabeled,
            anno_file=cfg.uanno_file if cfg.get("uanno_file", False) else None)

        transform_val = BaseTransform(cfg.vpipeline)
        test_dataset = base_dict[cfg.type](cfg.root,
                                        train=False,
                                        transform=transform_val,
                                        download=False)

    return train_labeled_dataset, train_unlabeled_dataset, test_dataset


Overwriting ./Classification-SemiCLS/dataset/builder.py


In [ ]:
%%writefile ./Classification-SemiCLS/train_semi.py  

""" The Code is under Tencent Youtu Public Rule

this is the mian sript in our SSL tool box
The interface contains many args

$CONFIG_PATH=/your/config/path
$OUTPUT_PATH=/your/output/path
For single gpu
number_of_gpus=1
python3 train_semi.py --cfg $CONFIG_PATH  --out $OUTPUT_PATH --seed 5

For multi gpu
number_of_gpus=N
python3 -m torch.distributed.launch --nproc_per_node $number_of_gpus \
    train_semi.py\
        --cfg $CONFIG_PATH
        --out $OUTPUT_PATH --use_BN True  --seed 5

"""
import argparse
import logging
import math
import os
import random
import time
from calendar import c
from sched import scheduler

import numpy as np
import torch
import torch.nn.functional as F
from mmcv import Config
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from dataset import builder as dataset_builder
from models import builder as model_builder
from optimizer import builder as optim_builder
from scheduler import builder as scheduler_builder
from trainer import builder as trainer_builder
from utils import AverageMeter, AverageMeterManeger, accuracy
from utils.ckpt_utils import save_ckpt_dict
from utils.config_utils import overwrite_config
from utils.log_utils import get_default_logger

torch.autograd.set_detect_anomaly(True)

# global variables
global logger
best_acc = 0
SCALER = None


def set_seed(args):
    """ set seed for the whole program for removing randomness
    """
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def get_test_model(ema_model, model, use_ema):
    """ use ema model or test model
    """
    if use_ema:
        test_model = ema_model.ema
        test_prefix = "ema"
    else:
        test_model = model
        test_prefix = ""
    return test_model, test_prefix


def main():
    args = get_args()
    global best_acc

    # prepare config and make output dir
    cfg = Config.fromfile(args.cfg)
    cfg = overwrite_config(cfg, args.other_args)
    cfg.resume = args.resume
    cfg.data['eval_step'] = cfg.train.eval_step

    # set amp scaler, usually no use
    global SCALER
    if args.fp16:
        SCALER = torch.cuda.amp.GradScaler()
    else:
        SCALER = None

    # set summary writer on rank 0
    if args.local_rank in [-1, 0]:
        os.makedirs(args.out, exist_ok=True)
        args.writer = SummaryWriter(args.out)
        cfg.dump(os.path.join(args.out, os.path.basename(args.cfg)))

    # set up logger
    global logger
    logger = get_default_logger(
        args=args,
        logger_name='root',
        default_level=logging.INFO
        if args.local_rank in [-1, 0] else logging.WARN,
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        rank=args.local_rank)

    #set random seed
    if args.seed is not None:
        cfg.seed = args.seed
    elif cfg.get("seed", None) is not None:
        args.seed = cfg.seed

    # set folds for stl10 dataset if used
    if "folds" in cfg.data.keys():
        cfg.data.folds = cfg.seed

    args.amp = False
    if cfg.get("amp", False) and cfg.amp.use:
        args.amp = True
        args.opt_level = cfg.amp.opt_level

    if not args.pretrained and "pretrained" in cfg.keys():
        args.pretrained = cfg.pretrained

    args.total_steps = cfg.train.total_steps
    args.eval_steps = cfg.train.eval_step

    # init dist params
    if args.local_rank == -1:
        device = torch.device('cuda', args.gpu_id)
        args.world_size = 1
        args.n_gpu = torch.cuda.device_count()
    else:
        torch.cuda.set_device(args.local_rank)
        device = torch.device('cuda', args.local_rank)
        torch.distributed.init_process_group(backend='nccl')
        args.world_size = torch.distributed.get_world_size()
        args.n_gpu = 1

    # set device
    args.device = device

    logger.warning(
        f"Process rank: {args.local_rank}, "
        f"device: {args.device}, "
        f"n_gpu: {args.n_gpu}, "
        f"distributed training: {bool(args.local_rank != -1)}, "
        f"16-bits training: {args.amp}", )

    logger.info(dict(args._get_kwargs()))

    if args.seed is not None:
        set_seed(args)

    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()

    # make dataset
    #labeled_dataset, unlabeled_dataset, test_dataset = dataset_builder.build(cfg.data)
    labeled_dataset, unlabeled_dataset, test_dataset = dataset_builder.build(
        cfg.data)

    if args.local_rank == 0:
        torch.distributed.barrier()

    # make dataset loader
    train_sampler = RandomSampler if args.local_rank == -1 else DistributedSampler

    labeled_trainloader, unlabeled_trainloader, test_loader = get_dataloader(
        cfg, train_sampler, labeled_dataset, unlabeled_dataset, test_dataset)

    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()

    model = model_builder.build(cfg.model)

    # load from pre-trained, before DistributedDataParallel constructor
    if args.pretrained:
        if os.path.isfile(args.pretrained):
            print("=> loading checkpoint '{}'".format(args.pretrained))
            checkpoint = torch.load(args.pretrained, map_location="cpu")

            # rename moco pre-trained keys
            state_dict = checkpoint['state_dict']
            for k in list(state_dict.keys()):
                # retain only encoder_q up to before the embedding layer
                if k.startswith('module.encoder_q'
                                ) and not k.startswith('module.encoder_q.fc'):
                    # remove prefix
                    state_dict[k[len("module.encoder_q."):]] = state_dict[k]
                # delete renamed or unused k
                del state_dict[k]

            msg = model.load_state_dict(state_dict, strict=False)
            print("Missing keys", msg.missing_keys)

            print("=> loaded pre-trained model '{}'".format(args.pretrained))
        else:
            print("=> no checkpoint found at '{}'".format(args.pretrained))

    if args.local_rank == 0:
        torch.distributed.barrier()

    model.to(args.device)

    # make optimizer,scheduler
    optimizer = optim_builder.build(cfg.optimizer, model)
    scheduler = set_scheduler(cfg, args, optimizer)

    # set ema
    args.use_ema = False
    ema_model = None
    if cfg.get("ema", False) and cfg.ema.use:
        args.use_ema = True
        from models.ema import ModelEMA
        ema_model = ModelEMA(args.device, model, cfg.ema.decay)

    args.start_epoch = 0

    # initialize from resume for fixed info and task_specific_info
    task_specific_info = dict()

    if args.resume:
        if args.use_ema:
            resume(args,
                   model,
                   optimizer,
                   scheduler,
                   task_specific_info,
                   ema_model=ema_model)
        else:
            resume(args, model, optimizer, scheduler, task_specific_info)

    # builde model trainer
    cfg.train.trainer['amp'] = args.amp
    model_trainer = trainer_builder.build(cfg.train.trainer)(device=device,
                                                             all_cfg=cfg)

    # process model
    if args.amp:
        from apex import amp
        model, optimizer = amp.initialize(model,
                                          optimizer,
                                          opt_level=args.opt_level)
    #use_BN
    use_batchnorm = args.use_BN
    if use_batchnorm:
        model = nn.SyncBatchNorm.convert_sync_batchnorm(model)

    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model,
            device_ids=[args.local_rank],
            output_device=args.local_rank,
            find_unused_parameters=True)

    log_info(cfg, args)

    model.zero_grad()
    #train loop
    train(args, cfg, labeled_trainloader, unlabeled_trainloader, test_loader,
          model, optimizer, ema_model, scheduler, model_trainer,
          task_specific_info)


# resume from checkpoint
def resume(args,
           model,
           optimizer,
           scheduler,
           task_specific_info,
           ema_model=None):
    logger.info("==> Resuming from checkpoint..")
    if not os.path.isfile(args.resume):
        logger.info("Error resuming from {}! try resume from last one".format(
            args.resume))
        args.resume = os.path.join(args.out, "checkpoint.pth.tar")
        logger.info("Resuming from {}".format(args.resume))

    # try resume if specified
    if not os.path.isfile(args.resume):
        logger.info("Failed to resume from {}".format(args.resume))
    else:
        args.out = os.path.dirname(args.resume)
        checkpoint = torch.load(args.resume)
        best_acc = checkpoint['best_acc']
        args.start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        if args.use_ema:
            ema_model.ema.load_state_dict(checkpoint['ema_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])

        for key in checkpoint.keys():
            if key not in [
                    'epoch', 'state_dict', 'ema_state_dict', 'acc', 'best_acc',
                    'optimizer', 'scheduler'
            ]:
                task_specific_info[key] = checkpoint[key]
                try:
                    task_specific_info[key] = task_specific_info[key].to(
                        args.device)
                except:
                    pass


#set scheduler
def set_scheduler(cfg, args, optimizer):
    args.epochs = math.ceil(cfg.train.total_steps / cfg.train.eval_step)
    args.eval_step = cfg.train.eval_step
    args.total_steps = cfg.train.total_steps
    scheduler = scheduler_builder.build(cfg.scheduler)(optimizer=optimizer)

    return scheduler


# log info before training
def log_info(cfg, args):
    logger.info("***** Running training *****")
    logger.info(f"  Task = {cfg.data.type}")
    if "num_labeled" in cfg.data.keys():
        logging_num_labeled = cfg.data.num_labeled
    elif "percent" in cfg.data.keys():
        logging_num_labeled = "{}%".format(cfg.data.percent)
    else:
        logging_num_labeled = "seed {}".format(cfg.seed)

    logger.info(f"  Num Label = {logging_num_labeled}")
    logger.info(f"  Num Epochs = {args.epochs}")
    logger.info(f"  Batch size per GPU = {cfg.data.batch_size}")
    logger.info(
        f"  Total train batch size = {cfg.data.batch_size*args.world_size}")
    logger.info(f"  Total optimization steps = {args.total_steps}")
    logger.info(cfg)


# get args
def get_args():
    parser = argparse.ArgumentParser(description='PyTorch FixMatch Training')
    parser.add_argument('--cfg', type=str, required=True, help='a config')
    parser.add_argument('--gpu-id',
                        default='0',
                        type=int,
                        help='id(s) for CUDA_VISIBLE_DEVICES')
    parser.add_argument('--out',
                        default='result',
                        help='directory to output the result')
    parser.add_argument('--pretrained',
                        default=None,
                        help='directory to pretrained model')
    parser.add_argument('--resume',
                        default='',
                        type=str,
                        help='path to latest checkpoint (default: none)')
    parser.add_argument('--seed', default=None, type=int, help="random seed")
    parser.add_argument('--use_BN',
                        default=False,
                        type=bool,
                        help="use_batchnorm")
    parser.add_argument('--fp16',
                        action='store_true',
                        help="whether use fp16 for training")
    parser.add_argument("--local_rank",
                        type=int,
                        default=-1,
                        help="For distributed training: local_rank")
    parser.add_argument('--no-progress',
                        action='store_true',
                        help="don't use progress bar")
    parser.add_argument(
        '--other-args',
        default='',
        type=str,
        help='other args to overwrite the config, keys are split \
                            by space and args split by |, such as \'seed 1|train trainer T 1|\' '
    )

    args = parser.parse_args()
    return args


# labeled_trainloader,unlabeled_trainloader,test_loader
def get_dataloader(cfg, train_sampler, labeled_dataset, unlabeled_dataset,
                   test_dataset):
    # prepare labeled_trainloader
    labeled_trainloader = DataLoader(labeled_dataset,
                                     sampler=train_sampler(labeled_dataset),
                                     batch_size=cfg.data.batch_size,
                                     num_workers=cfg.data.num_workers,
                                     drop_last=True)
    # prepare unlabeled_trainloader
    unlabeled_trainloader = DataLoader(
        unlabeled_dataset,
        sampler=train_sampler(unlabeled_dataset),
        batch_size=cfg.data.batch_size * cfg.data.mu,
        num_workers=cfg.data.num_workers,
        drop_last=True)
    # prepare test_loader
    test_loader = DataLoader(test_dataset,
                             sampler=SequentialSampler(test_dataset),
                             batch_size=cfg.data.batch_size,
                             num_workers=cfg.data.num_workers)

    return labeled_trainloader, unlabeled_trainloader, test_loader


# train_loop
def train(args, cfg, labeled_trainloader, unlabeled_trainloader, test_loader,
          model, optimizer, ema_model, scheduler, model_trainer,
          task_specific_info):
    """
    Train function for training
    """
    global best_acc
    test_accs = []

    if args.world_size > 1:
        labeled_epoch = 0
        unlabeled_epoch = 0
        labeled_trainloader.sampler.set_epoch(labeled_epoch)
        unlabeled_trainloader.sampler.set_epoch(unlabeled_epoch)

    labeled_iter = iter(labeled_trainloader)
    unlabeled_iter = iter(unlabeled_trainloader)

    for epoch in range(args.start_epoch, args.epochs):
        # init logger
        meter_manager = AverageMeterManeger()
        meter_manager.register('batch_time')
        meter_manager.register('data_time')
        end = time.time()
        model.train()
        if not args.no_progress:
            # p_bar = tqdm(range(args.eval_step),
            #              disable=args.local_rank not in [-1, 0])
            pass
        for batch_idx in range(args.eval_step):
            try:
                data_x = next(labeled_iter)
            except Exception:
                if args.world_size > 1:
                    labeled_epoch += 1
                    labeled_trainloader.sampler.set_epoch(labeled_epoch)
                labeled_iter = iter(labeled_trainloader)
                data_x = next(labeled_iter)

            try:
                data_u = next(unlabeled_iter)
            except Exception:
                if args.world_size > 1:
                    unlabeled_epoch += 1
                    unlabeled_trainloader.sampler.set_epoch(unlabeled_epoch)
                unlabeled_iter = iter(unlabeled_trainloader)
                data_u = next(unlabeled_iter)

            meter_manager.data_time.update(time.time() - end)
            # calculate loss
            loss_dict = model_trainer.compute_loss(
                data_x=data_x,
                data_u=data_u,
                model=model,
                optimizer=optimizer,
                epoch=epoch,
                iter=batch_idx,
                ema_model=ema_model,
                task_specific_info=task_specific_info,
                SCALER=SCALER if SCALER is not None else None)

            # update logger
            meter_manager.try_register_and_update(loss_dict)

            # step
            if SCALER is not None:
                SCALER.step(optimizer)
            else:
                optimizer.step()
            scheduler.step()

            # Updates the scale for next iteration
            if SCALER is not None:
                SCALER.update()

            # update ema if needed
            if args.use_ema:
                ema_model.update(model)
            model.zero_grad()

            meter_manager.batch_time.update(time.time() - end)
            end = time.time()
            if not args.no_progress and args.local_rank in [-1, 0]:
                if batch_idx % cfg.log.interval == 0:
                    meter_desc = meter_manager.get_desc()
                    logger.info(
                        "Train Epoch: {epoch}/{epochs:4}. Iter: {batch:4}/{iter:4}. LR: {lr:.4f} {desc}"
                        .format(epoch=epoch + 1,
                                epochs=args.epochs,
                                batch=batch_idx + 1,
                                iter=args.eval_step,
                                lr=scheduler.get_last_lr()[0],
                                desc=meter_desc))
                    # p_bar.update()

        # if not args.no_progress:
        #     p_bar.close()

        if args.local_rank in [-1, 0]:
            meter_manager.add_to_writer(args.writer, epoch, prefix="train/")

            # add test info
            test_model, test_prefix = get_test_model(ema_model, model,
                                                     args.use_ema)
            test_loss, test_acc = test(args, test_loader, test_model, epoch)
            args.writer.add_scalar('test/1.{}_acc'.format(test_prefix),
                                   test_acc, epoch)
            args.writer.add_scalar('test/2.{}_loss'.format(test_prefix),
                                   test_loss, epoch)
            is_best = test_acc > best_acc
            best_acc = max(test_acc, best_acc)

            # add second evaluatino writer needed
            if cfg.get("evaluation", False) and cfg.evaluation.eval_both:
                test_model_second, test_prefix_second = get_test_model(
                    ema_model, model, not args.use_ema)
                test_loss_second, test_acc_second = test(
                    args, test_loader, test_model_second, epoch)
                args.writer.add_scalar(
                    'test/1.{}_acc'.format(test_prefix_second),
                    test_acc_second, epoch)
                args.writer.add_scalar(
                    'test/2.{}_loss'.format(test_prefix_second),
                    test_loss_second, epoch)

            # save model
            if epoch % cfg.ckpt.interval == 0:
                save_ckpt_dict(args, model, ema_model, epoch, test_acc,
                               optimizer, scheduler, task_specific_info,
                               is_best, best_acc)

            test_accs.append(test_acc)
            logger.info('Best top-1 acc: {:.2f}'.format(best_acc))
            logger.info('Mean top-1 acc: {:.2f}\n'.format(
                np.mean(test_accs[-20:])))

    # save last ckpt
    if args.local_rank in [-1, 0]:
        save_ckpt_dict(args, model, ema_model, epoch, test_acc, optimizer,
                       scheduler, task_specific_info, is_best, best_acc)

    if args.local_rank in [-1, 0]:
        args.writer.close()


#test/valiadate step


def test(args, test_loader, model, epoch):
    """ Test function for model and loader
        when the model is ema model, will test the ema model
        when the model is model, will test the regular model
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    if not args.no_progress:
        test_loader = tqdm(test_loader, disable=args.local_rank not in [-1, 0])

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            data_time.update(time.time() - end)
            model.eval()

            inputs = inputs.to(args.device)
            targets = targets.to(args.device)
            outputs = model(inputs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            loss = F.cross_entropy(outputs, targets)

            prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
            losses.update(loss.item(), inputs.shape[0])
            top1.update(prec1.item(), inputs.shape[0])
            top5.update(prec5.item(), inputs.shape[0])
            batch_time.update(time.time() - end)
            end = time.time()
            if not args.no_progress:
                test_loader.set_description(
                    "Test Iter: {batch:4}/{iter:4}. Data: {data:.3f}s. \
                    Batch: {bt:.3f}s. Loss: {loss:.4f}. top1: {top1:.2f}. top5: {top5:.2f}. "
                    .format(
                        batch=batch_idx + 1,
                        iter=len(test_loader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        loss=losses.avg,
                        top1=top1.avg,
                        top5=top5.avg,
                    ))
        if not args.no_progress:
            test_loader.close()

    logger.info("Epoch {} top-1 acc: {:.2f}".format(epoch, top1.avg))
    logger.info("Epoch {} top-5 acc: {:.2f}".format(epoch, top5.avg))
    return losses.avg, top1.avg


if __name__ == '__main__':
    main()


Overwriting ./Classification-SemiCLS/train_semi.py


### Run the training

In [ ]:
#Running UCM with Fixmatch baseline
!python3 ./Classification-SemiCLS/train_semi.py --cfg ./Classification-SemiCLS/configs/fm_ucm.py --gpu-id 0 --out ./Classification-SemiCLS/results/fixmatch/fm_ucm --seed 5 

/content/drive/MyDrive/GeoPythonWorkshop_SSL/Classification-SemiCLS/mmcv/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
2023-03-05 20:14:16.244943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 20:14:18.933016: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared ob